# Data Preparation Pipeline
The goal of this notebook is to process the raw rasters through a pipeline that selects, cleans, and prepares data to be ready to trained. The first step is to ensure relevant dependencies are imported:

In [1]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.transform import from_origin
import numpy as np
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import re

The raw GeoTIFFs all have different coordinate reference systems, but GURS uses WGS 84. Therefore, we need to convert all the imagery to EPSG:4326. Furthermore, to help the model digest the imagery more easily, we need to avoid loading whole GeoTIFFs into RAM at a time. For any raw GeoTIFF, we can slice the raster into 64 quadrants to enhance surface detail. Some tiles will be mostly black. If more than 20% of the tile is black pixels, we discard it. Otherwise, we save the output into a new directory on disk.

In [2]:
tif_list = glob.glob("HLS2-s30_rgb_raw/*_[0-9][0-9].tif")
counter = 0    

for tif_path in tqdm(tif_list, desc="Generating tiles"):
    with rasterio.open(tif_path) as src:
        transform, width, height = calculate_default_transform(src.crs, "EPSG:4326", src.width, src.height, *src.bounds)
        profile = src.profile.copy()
        profile.update({"crs": "EPSG:4326", "transform": transform, "width": width, "height": height})
        data = np.zeros((src.count, height, width), dtype=src.dtypes[0])
        
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=data[i - 1],
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs="EPSG:4326",
                resampling=Resampling.nearest
            )

        # Split into 8×8 grid
        rows, cols = 8, 8
        h_step = height // rows
        w_step = width // cols

        for i in range(rows):
            for j in range(cols):
                quad = data[:, i * h_step:(i + 1) * h_step, j * w_step:(j + 1) * w_step]
                black_pixels = np.all(quad == 0, axis=0)
                
                if black_pixels.sum() / black_pixels.size > 0.2:  # discard quadrant if >20% black
                    continue

                outfile = f"HLS2-s30_rgb/tile{counter}.tif"
                counter += 1
                quad_transform = from_origin(transform[2] + j * w_step * transform[0], transform[5] + i * h_step * transform[4], transform[0], transform[4])
                out_profile = profile.copy()
                out_profile.update({"height": quad.shape[1], "width": quad.shape[2], "count": quad.shape[0], "dtype": quad.dtype, "transform": quad_transform})

                with rasterio.open(outfile, "w", **out_profile) as dst:
                    dst.write(quad)

Generating tiles: 100%|███████████████████████████████████████████████████████████████| 600/600 [33:11<00:00,  3.32s/it]


## Ground Truth Determenation Methodology
The ground truths of each tile will be extracted from the GURS 2020 raster in two main steps:
1. Isolate the portion of GURS 2020 such that it represents the geographical area covered by any specific tile.
2. Assign a score to each land-cover label in the GURS dataset (`urban`, `rural`, `uninhabited`), reflecting the fractional representation of the respective class within each region.

### I. Extract raster window of GURS corresponding to each tile.
Let the training tile bounds be defined as four latitude and longitude pairs representing the four corners of the bounding box (provided by `training_tile.bounds`). We can denote this as $UL, UR, LL, LR$. The objective of this step is to find the rows and columns in GURS corresponding to these bounds ($ x\in [x_0, x_f]$, $y \in [y_0, y_f]$). 

We only need two degrees of freedom ($UL, LR$) to find the full bounds. Using `rasterio`, we can read the `GURS.transform` attribute get the affine transformation:

$$
\mathbf{T} =
\begin{bmatrix}
0 & 0 & -180 \\
0  & -0 & 77.91 \\
0 & 0 & 1
\end{bmatrix}.
$$

Apply the inverse to get row and column bounds:

$$
\mathbf{T}^{-1}UL = \begin{bmatrix}
x_0 \\
y_0 \\
1
\end{bmatrix},
$$$$
\mathbf{T}^{-1}LR = \begin{bmatrix}
x_f \\
y_f \\
1
\end{bmatrix},
$$

where $y$ represents the rows and $x$ represents the columns. An additional processing step is needed to ensure $x, y \in \mathbb{Z}$:

$$
x_0 = ⌊\min(x_0, x_f)⌋, \,\,\,\, x_f = ⌊\max(x_0, x_f)⌋,
$$$$
y_0 = ⌊\min(y_0, y_f)⌋, \,\,\,\, y_f = ⌊\max(y_0, y_f)⌋.
$$

Let $GURS_{wnd} = \text{GURS}[y_0:y_f, x_0:x_f]$ represent the slice of the $\text{GURS}$ raster at the defined bounds. After following these steps above, we get $GURS_{wnd} \in \{1, 2, 127\}^{126 \times 148}$. This is slightly smaller than the training tile's dimensions because of difference in resolution.

### II. Compute frequency of each class for every tile.
This algorithm will get the relative weighted frequencies of each value in $GURS_{wnd}$. Define a binary mask for a value $n$:

$$
\mathcal{M}^{(n)}(i, j) = \begin{cases} 
      1, & GURS_{wnd}[i][j] = n \\
      0 & \text{otherwise}
   \end{cases}
$$

To get the frequency of any value $n$, define

$$
\mathcal{F}(n) = \frac{\sum_{i=y_0}^{y_f} \sum_{j=x_0}^{x_f} \mathcal{M}^{(n)}(i, j)}{126 \times 148 = 18648}.
$$

Next, the weights of each inhabited class are set to $100/9$. This is because each GURS pixel is $100 \times 100 = 10000 \text{ m}^2$ whereas the tile pixels are only $30 \times 30 = 900 \text{ m}^2$. The ratio of the weights represents the size differences of the pixel. Uninhabited pixels do not get weighted as the population density is negligible. We use the following function to ensure normalized weighted frequencies:

$$
\mathcal{F}_w(n) = \frac{w_n \mathcal{F}(n)}{\sum_{i \in \{1, 2, 127\}} w_i \mathcal{F}(i)}.
$$

In practice, rather than looping naively, we can use a vectorized process to get the entire frequency distribution in one pass:

$$
\vec{F}_w = \frac{\vec{w} \odot \vec{f}}{\vec{1} (\vec{w} \odot \vec{f})},
$$
where $\odot$ is the Hadamard product operator, $\vec{1} \in \mathbb{R}^3$ is the vector of all ones, $\vec{w} = \begin{bmatrix} w_1 & w_2 & w_{127} \end{bmatrix}^\top$, and $\vec{f} = \begin{bmatrix} \mathcal{F}(1) & \mathcal{F}(2) & \mathcal{F}(127) \end{bmatrix}^\top$. Finally, the frequencies are saved as a `.csv` file representing the ground truths of each tile.

In [3]:
label_decoder = {1: "urban", 2: "rural", 127: "uninhabited"}
weights = np.array([100 / 9, 100 / 9, 1])
rows = []

with rasterio.open("GURS_2020.tif") as GURS:
    for geotiff in tqdm(sorted(glob.glob("HLS2-s30_rgb/tile*.tif"), key=lambda x: int(re.search(r'tile(\d+)\.tif', x).group(1))), desc="Processing tiles"):
        train_index = int(geotiff.split("tile")[-1].split(".tif")[0]) # extract train index from filename
        with rasterio.open(geotiff) as training_tile:
            # compute window in GURS
            col_start, row_start = ~GURS.transform * (training_tile.bounds.left, training_tile.bounds.top)
            col_stop, row_stop = ~GURS.transform * (training_tile.bounds.right, training_tile.bounds.bottom)
            row_start, row_stop = int(min(row_start, row_stop)), int(max(row_start, row_stop))
            col_start, col_stop = int(min(col_start, col_stop)), int(max(col_start, col_stop))
            GURS_wnd = GURS.read(1, window=((row_start, row_stop), (col_start, col_stop))) 

            # SHORTCUT: if all values are the same
            if np.min(GURS_wnd) == np.max(GURS_wnd): 
                frequencies = np.zeros(3)
                label = np.min(GURS_wnd)
                frequencies[list(label_decoder.keys()).index(label)] = 1
            else: 
                # compute score based on weighted frequency
                freqs = np.array([(GURS_wnd == n).sum() for n in [1, 2, 127]], dtype=float)
                freqs *= weights
                frequencies = freqs / freqs.sum()
                    
            rows.append({label_decoder[n]: freq for n, freq in zip([1, 2, 127], frequencies)})

df = pd.DataFrame(rows)
df.to_csv("ground_truths.csv", index=False)

Processing tiles: 100%|██████████████████████████████████████████████████████████| 30444/30444 [00:33<00:00, 905.44it/s]
